In [ ]:
import numpy as np
import pandas as pd
import math

import itertools
from datetime import datetime
import time

from finrl.meta.env_cryptocurrency_trading.env_multiple_crypto import CryptoEnv

from finrl.agents.stablebaselines3.drl_agent import DRLAgent
from finrl.meta.data_processor import DataProcessor

from lib.drl import load_dataset, data_split
from config import general as config
from config import crypto
# Plotting
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates

In [ ]:
def train(start_date, end_date, ticker_list, data_source, time_interval, technical_indicator_list, env, model_name, if_vix=True, **kwargs):
    # process data using unified data processor
    # dp = DataProcessor("file", filename="datasets/crypto/crypto_1h.csv", **kwargs)
    # dp.download_data(ticket_list, start_date, end_date, '1h')
    # price_array, tech_array, turbulence_array = DP.run(ticker_list, technical_indicator_list, if_vix, cache=True)

    data_config = {'price_array': price_array, 'tech_array': tech_array, 'turbulence_array': turbulence_array}

    # build environment using processed data
    env_instance = env(config=data_config)

    # read parameters and load agents
    current_working_dir = kwargs.get('current_working_dir', './' + str(model_name))

    total_timesteps = kwargs.get('total_timesteps', 1e6)
    agent_params = kwargs.get('agent_params')

    agent = DRLAgent(env=env_instance)

    model = agent.get_model(model_name, model_kwargs=agent_params)
    trained_model = agent.train_model(model=model, tb_log_name=model_name, total_timesteps=total_timesteps)
    
    print('Training finished!')
    trained_model.save(current_working_dir)
    print('Trained model saved in ' + str(current_working_dir))

In [ ]:
def test(start_date, end_date, ticker_list, data_source, time_interval, technical_indicator_list, drl_lib, env, model_name, if_vix=True, **kwargs):
    # process data using unified data processor
    # dp = DataProcessor(data_source, start_date, end_date, time_interval, **kwargs)
    # price_array, tech_array, turbulence_array = dp.run(ticker_list, technical_indicator_list, if_vix, cache=True)

    np.save('./price_array.npy', price_array)
    data_config = {'price_array': price_array, 'tech_array': tech_array, 'turbulence_array': turbulence_array}
    # build environment using processed data
    env_instance = env(config=data_config)

    env_config = { "price_array": price_array, "tech_array": tech_array, "turbulence_array": turbulence_array, "if_train": False }
    env_instance = env(config=env_config)

    # load elegantrl needs state dim, action dim and net dim
    net_dimension = kwargs.get("net_dimension", 2 ** 7)
    current_working_dir = kwargs.get("current_working_dir", "./" + str(model_name))
    print("price_array: ", len(price_array))

    episode_total_assets = DRLAgent.DRL_prediction_load_from_file(model_name=model_name, environment=env_instance, cwd=current_working_dir)
    return episode_total_assets

def get_time():
    now = datetime.now()
    return now.strftime("%d.%m.%Y %H:%M:%S")
    
def get_duration(duration):
    m, s = divmod(duration, 60)
    h, m = divmod(m, 60)
    return f'{h:02.0f}:{m:02.0f}:{s:02.0f}'

In [ ]:
TICKER_LIST = ['BTCUSDT', 'ETHUSDT', 'ADAUSDT', 'BNBUSDT', 'XRPUSDT', 'SOLUSDT', 'DOTUSDT', 'DOGEUSDT', 'AVAXUSDT', 'UNIUSDT']
# INDICATORS = ['macd', 'rsi_30', 'cci_30', 'dx_30']
# TRAIN_START_DATE = '2020-10-01'
# TRAIN_END_DATE = '2022-01-01'
# TEST_START_DATE = '2022-01-01'
# TEST_END_DATE = '2022-07-01'

In [ ]:
# df = load_dataset("datasets/crypto/crypto_1d.csv", INDICATORS, True, False, '1d')
df = pd.read_csv(f"{config.DATA_SAVE_DIR}/crypto/crypto_1h_parsed.csv", index_col=0)
df_train = data_split(df, crypto.TRAIN_START_DATE, crypto.TRAIN_END_DATE)
df_test = data_split(df, crypto.TEST_START_DATE, crypto.TEST_END_DATE)
print(f"train {df_train.shape}")
print(f"test  {df_test.shape}")
# df.to_csv('crypto_1d_parsed.csv')

In [ ]:
env = CryptoEnv

In [ ]:
start = time.time()
dur = '1min'
df = load_dataset(f"datasets/crypto/crypto_{dur}.csv", crypto.INDICATORS, True, False, dur)
print(f"{get_time()} : Loading done after {get_duration(time.time()-start)}")
df.to_csv(f"datasets/crypto/crypto_{dur}_parsed.csv")
print(f"{get_time()} : Saved after {get_duration(time.time()-start)}")

In [ ]:
duration = time.time() - start
m, s = divmod(duration, 60)
h, m = divmod(m, 60)


In [ ]:
s = 5

In [ ]:
# Training
train(start_date=TRAIN_START_DATE, end_date=TRAIN_END_DATE, ticker_list=TICKER_LIST,
      data_source='binance', time_interval='5m', technical_indicator_list=INDICATORS,
      env=env, model_name='ppo', current_working_dir='./test_ppo', erl_params=ERL_PARAMS,
      break_step=5e4, if_vix=False)

In [ ]:
# Testing
account_value = test(start_date=TEST_START_DATE, end_date=TEST_END_DATE, ticker_list=TICKER_LIST,
                     data_source='binance', time_interval='5m', technical_indicator_list=INDICATORS,
                     drl_lib='elegantrl', env=env, model_name='ppo', current_working_dir='./test_ppo',
                     net_dimension=2 ** 9, if_vix=False)

In [ ]:
# calculate agent returns
account_value = np.array(account_value)
agent_returns = account_value / account_value[0]

# calculate buy-and-hold btc returns
price_array = np.load('./price_array.npy')
btc_prices = price_array[:, 0]
buy_hold_btc_returns = btc_prices / btc_prices[0]

# calculate equal weight portfolio returns
price_array = np.load('./price_array.npy')
initial_prices = price_array[0, :]
equal_weight = np.array([1e5 / initial_prices[i] for i in range(len(TICKER_LIST))])
equal_weight_values = []
for i in range(0, price_array.shape[0]):
    equal_weight_values.append(np.sum(equal_weight * price_array[i]))
equal_weight_values = np.array(equal_weight_values)
equal_returns = equal_weight_values / equal_weight_values[0]

In [ ]:
# plot
plt.figure(dpi=200)
plt.grid()
plt.grid(which='minor', axis='y')
plt.title('Cryptocurrency Trading ', fontsize=20)
plt.plot(agent_returns, label='StableBaselines Agent', color='red')
plt.plot(buy_hold_btc_returns, label='Buy-and-Hold BTC', color='blue')
plt.plot(equal_returns, label='Equal Weight Portfolio', color='green')
plt.ylabel('Return', fontsize=16)
plt.xlabel('Times (5min)', fontsize=16)
plt.xticks(size=14)
plt.yticks(size=14)

'''ax = plt.gca()
ax.xaxis.set_major_locator(ticker.MultipleLocator(210))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(21))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.005))
ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1, decimals=2))
ax.xaxis.set_major_formatter(ticker.FixedFormatter([]))'''

plt.legend(fontsize=10.5)